In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.0803, train_loss: 2.18369055, train_accuracy: 0.3828, test_Accuracy: 0.2247
Epoch: [ 0] [    1/  468] time: 1.3111, train_loss: 2.15163136, train_accuracy: 0.4141, test_Accuracy: 0.3786
Epoch: [ 0] [    2/  468] time: 1.5630, train_loss: 2.08479357, train_accuracy: 0.4922, test_Accuracy: 0.4720
Epoch: [ 0] [    3/  468] time: 1.8300, train_loss: 2.00652504, train_accuracy: 0.5312, test_Accuracy: 0.5061
Epoch: [ 0] [    4/  468] time: 2.0709, train_loss: 1.95948124, train_accuracy: 0.5391, test_Accuracy: 0.5594
Epoch: [ 0] [    5/  468] time: 2.3244, train_loss: 1.83183742, train_accuracy: 0.6719, test_Accuracy: 0.6205
Epoch: [ 0] [    6/  468] time: 2.5604, train_loss: 1.73813307, train_accuracy: 0.7031, test_Accuracy: 0.6555
Epoch: [ 0] [    7/  468] time: 2.8121, train_loss: 1.58963156, train_accuracy: 0.7734, test_Accuracy: 0.6845
Epoch: [ 0] [    8/  468] time: 3.0389, train_loss: 1.59636617, train_accuracy: 0.6641, test_Accuracy: 0.7155
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 19.6260, train_loss: 0.29907966, train_accuracy: 0.9219, test_Accuracy: 0.8999
Epoch: [ 0] [   76/  468] time: 19.8798, train_loss: 0.30475283, train_accuracy: 0.9062, test_Accuracy: 0.8892
Epoch: [ 0] [   77/  468] time: 20.1829, train_loss: 0.21419051, train_accuracy: 0.9453, test_Accuracy: 0.8886
Epoch: [ 0] [   78/  468] time: 20.4917, train_loss: 0.39487314, train_accuracy: 0.8828, test_Accuracy: 0.9003
Epoch: [ 0] [   79/  468] time: 20.7160, train_loss: 0.21239606, train_accuracy: 0.9531, test_Accuracy: 0.9052
Epoch: [ 0] [   80/  468] time: 20.9718, train_loss: 0.41592318, train_accuracy: 0.8828, test_Accuracy: 0.9088
Epoch: [ 0] [   81/  468] time: 21.2119, train_loss: 0.20283839, train_accuracy: 0.9531, test_Accuracy: 0.9086
Epoch: [ 0] [   82/  468] time: 21.6044, train_loss: 0.38184100, train_accuracy: 0.8516, test_Accuracy: 0.9067
Epoch: [ 0] [   83/  468] time: 21.8475, train_loss: 0.29717341, train_accuracy: 0.9141, test_Accuracy: 0.9088
E

Epoch: [ 0] [  149/  468] time: 38.4675, train_loss: 0.23141284, train_accuracy: 0.9375, test_Accuracy: 0.9308
Epoch: [ 0] [  150/  468] time: 38.7251, train_loss: 0.28845423, train_accuracy: 0.9141, test_Accuracy: 0.9325
Epoch: [ 0] [  151/  468] time: 38.9691, train_loss: 0.22665715, train_accuracy: 0.9609, test_Accuracy: 0.9326
Epoch: [ 0] [  152/  468] time: 39.2144, train_loss: 0.32931429, train_accuracy: 0.9141, test_Accuracy: 0.9327
Epoch: [ 0] [  153/  468] time: 39.4710, train_loss: 0.20734504, train_accuracy: 0.9297, test_Accuracy: 0.9308
Epoch: [ 0] [  154/  468] time: 39.7059, train_loss: 0.25172597, train_accuracy: 0.8984, test_Accuracy: 0.9302
Epoch: [ 0] [  155/  468] time: 39.9336, train_loss: 0.17872617, train_accuracy: 0.9375, test_Accuracy: 0.9283
Epoch: [ 0] [  156/  468] time: 40.2670, train_loss: 0.25558239, train_accuracy: 0.9141, test_Accuracy: 0.9265
Epoch: [ 0] [  157/  468] time: 40.5212, train_loss: 0.15190317, train_accuracy: 0.9688, test_Accuracy: 0.9244
E

Epoch: [ 0] [  223/  468] time: 56.9314, train_loss: 0.25231355, train_accuracy: 0.9453, test_Accuracy: 0.9432
Epoch: [ 0] [  224/  468] time: 57.1604, train_loss: 0.14486101, train_accuracy: 0.9609, test_Accuracy: 0.9435
Epoch: [ 0] [  225/  468] time: 57.4027, train_loss: 0.23587400, train_accuracy: 0.9062, test_Accuracy: 0.9450
Epoch: [ 0] [  226/  468] time: 57.6261, train_loss: 0.24113405, train_accuracy: 0.9375, test_Accuracy: 0.9453
Epoch: [ 0] [  227/  468] time: 57.9230, train_loss: 0.25196314, train_accuracy: 0.9219, test_Accuracy: 0.9449
Epoch: [ 0] [  228/  468] time: 58.1437, train_loss: 0.20395036, train_accuracy: 0.9453, test_Accuracy: 0.9444
Epoch: [ 0] [  229/  468] time: 58.4029, train_loss: 0.17502247, train_accuracy: 0.9531, test_Accuracy: 0.9443
Epoch: [ 0] [  230/  468] time: 58.6539, train_loss: 0.24831501, train_accuracy: 0.9453, test_Accuracy: 0.9438
Epoch: [ 0] [  231/  468] time: 58.9031, train_loss: 0.12744367, train_accuracy: 0.9531, test_Accuracy: 0.9441
E

Epoch: [ 0] [  297/  468] time: 74.9554, train_loss: 0.15658034, train_accuracy: 0.9453, test_Accuracy: 0.9508
Epoch: [ 0] [  298/  468] time: 75.1906, train_loss: 0.11178467, train_accuracy: 0.9531, test_Accuracy: 0.9509
Epoch: [ 0] [  299/  468] time: 75.4251, train_loss: 0.17607683, train_accuracy: 0.9609, test_Accuracy: 0.9510
Epoch: [ 0] [  300/  468] time: 75.6554, train_loss: 0.19688718, train_accuracy: 0.9453, test_Accuracy: 0.9524
Epoch: [ 0] [  301/  468] time: 75.8889, train_loss: 0.10318981, train_accuracy: 0.9766, test_Accuracy: 0.9531
Epoch: [ 0] [  302/  468] time: 76.1189, train_loss: 0.16841376, train_accuracy: 0.9531, test_Accuracy: 0.9512
Epoch: [ 0] [  303/  468] time: 76.4390, train_loss: 0.10988708, train_accuracy: 0.9688, test_Accuracy: 0.9504
Epoch: [ 0] [  304/  468] time: 76.7107, train_loss: 0.19259045, train_accuracy: 0.9297, test_Accuracy: 0.9496
Epoch: [ 0] [  305/  468] time: 76.9880, train_loss: 0.28796974, train_accuracy: 0.9219, test_Accuracy: 0.9500
E

Epoch: [ 0] [  371/  468] time: 93.1615, train_loss: 0.20624909, train_accuracy: 0.9453, test_Accuracy: 0.9582
Epoch: [ 0] [  372/  468] time: 93.4091, train_loss: 0.10977043, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  373/  468] time: 93.6528, train_loss: 0.15563942, train_accuracy: 0.9453, test_Accuracy: 0.9583
Epoch: [ 0] [  374/  468] time: 93.8840, train_loss: 0.08113511, train_accuracy: 0.9766, test_Accuracy: 0.9583
Epoch: [ 0] [  375/  468] time: 94.1072, train_loss: 0.25774193, train_accuracy: 0.8984, test_Accuracy: 0.9571
Epoch: [ 0] [  376/  468] time: 94.3597, train_loss: 0.14139314, train_accuracy: 0.9531, test_Accuracy: 0.9577
Epoch: [ 0] [  377/  468] time: 94.6049, train_loss: 0.15434693, train_accuracy: 0.9453, test_Accuracy: 0.9572
Epoch: [ 0] [  378/  468] time: 94.8482, train_loss: 0.06688659, train_accuracy: 0.9922, test_Accuracy: 0.9565
Epoch: [ 0] [  379/  468] time: 95.0776, train_loss: 0.22680070, train_accuracy: 0.9375, test_Accuracy: 0.9580
E

Epoch: [ 0] [  445/  468] time: 111.1967, train_loss: 0.08712028, train_accuracy: 0.9609, test_Accuracy: 0.9618
Epoch: [ 0] [  446/  468] time: 111.4773, train_loss: 0.07210135, train_accuracy: 0.9844, test_Accuracy: 0.9611
Epoch: [ 0] [  447/  468] time: 111.7873, train_loss: 0.10329678, train_accuracy: 0.9688, test_Accuracy: 0.9612
Epoch: [ 0] [  448/  468] time: 112.0299, train_loss: 0.12286036, train_accuracy: 0.9609, test_Accuracy: 0.9609
Epoch: [ 0] [  449/  468] time: 112.2720, train_loss: 0.14616615, train_accuracy: 0.9609, test_Accuracy: 0.9620
Epoch: [ 0] [  450/  468] time: 112.5464, train_loss: 0.09558084, train_accuracy: 0.9688, test_Accuracy: 0.9616
Epoch: [ 0] [  451/  468] time: 112.7747, train_loss: 0.07436172, train_accuracy: 0.9844, test_Accuracy: 0.9618
Epoch: [ 0] [  452/  468] time: 113.0223, train_loss: 0.17745319, train_accuracy: 0.9609, test_Accuracy: 0.9622
Epoch: [ 0] [  453/  468] time: 113.2555, train_loss: 0.12243685, train_accuracy: 0.9688, test_Accuracy:

# Test accuracy : 95.72%